In [57]:
%load_ext autoreload
%autoreload 2
    
import sys

sys.path.append("../lib/")

import pandas as pd

import prepare_dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
weather_train_path = "../data/weather_train.csv"

In [59]:
weather_train = pd.read_csv(weather_train_path, sep=";", index_col=0)

<ipython-input-9-3a06d231aa5a>:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_train = pd.read_csv(weather_train_path, sep=";", index_col=0)


In [60]:
prepare_dataset.prepare_dataset(weather_train)

         local_time    T     Po      P   Pa     U  \
0  31.12.2013 21:00 -1.5  758.9  773.2  0.7  99.0   
1  31.12.2013 18:00 -1.7  758.2  772.5  0.7  99.0   
2  31.12.2013 15:00 -1.7  757.5  771.8  0.0  99.0   
3  31.12.2013 12:00 -1.2  757.5  771.7  0.8  99.0   
4  31.12.2013 09:00 -1.2  756.7  770.9  1.2  99.0   

                                     DD   Ff  ff10  ff3  ...   Ch   VV   Td  \
0  Ветер, дующий с северо-северо-запада  2.0   NaN  NaN  ...  NaN  6.0 -1.6   
1         Ветер, дующий с северо-запада  3.0   NaN  NaN  ...  NaN  3.0 -1.9   
2         Ветер, дующий с северо-запада  4.0   NaN  NaN  ...  NaN  3.0 -1.9   
3         Ветер, дующий с северо-запада  3.0   NaN  NaN  ...  NaN  2.5 -1.4   
4                   Ветер, дующий с юга  3.0   NaN  NaN  ...  NaN  0.5 -1.4   

           RRR    tR    E  Tg   E'  sss station_id  
0          NaN   NaN  NaN NaN  NaN  NaN      34123  
1  Осадков нет  12.0  NaN NaN  NaN  NaN      34123  
2          NaN   NaN  NaN NaN  NaN  NaN      34

(              local_time    T     Po      P   Pa     U  \
 0       31.12.2013 21:00 -1.5  758.9  773.2  0.7  99.0   
 1       31.12.2013 18:00 -1.7  758.2  772.5  0.7  99.0   
 2       31.12.2013 15:00 -1.7  757.5  771.8  0.0  99.0   
 3       31.12.2013 12:00 -1.2  757.5  771.7  0.8  99.0   
 4       31.12.2013 09:00 -1.2  756.7  770.9  1.2  99.0   
 ...                  ...  ...    ...    ...  ...   ...   
 356501  01.01.2014 12:00  0.1  762.1  774.4  0.3  76.0   
 356502  01.01.2014 09:00  0.1  761.8  774.1  0.8  78.0   
 356503  01.01.2014 06:00 -0.1  761.0  773.3  0.0  78.0   
 356504  01.01.2014 03:00  0.4  761.0  773.3  0.7  81.0   
 356505  01.01.2014 00:00  0.6  760.3  772.6  0.8  89.0   
 
                                           DD   Ff  ff10  ff3  ...   Ch    VV  \
 0       Ветер, дующий с северо-северо-запада  2.0   NaN  NaN  ...  NaN   6.0   
 1              Ветер, дующий с северо-запада  3.0   NaN  NaN  ...  NaN   3.0   
 2              Ветер, дующий с северо-запада  

In [61]:
x, y = _

In [63]:
y

,VETER,SHKVAL,METEL,DOZD,SNEG,GRAD,TUMAN,GOLOLED
0,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,True,True
3,False,False,False,False,False,False,True,True
4,False,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...
356501,False,False,False,False,False,False,False,False
356502,False,False,False,False,False,False,False,False
356503,False,False,False,False,False,False,False,False
356504,False,False,False,False,False,False,False,False


In [47]:
weather_train.columns

Index(['local_time', 'T', 'Po', 'P', 'Pa', 'U', 'DD', 'Ff', 'ff10', 'ff3', 'N',
       'WW', 'W1', 'W2', 'Tn', 'Tx', 'Cl', 'Nh', 'H', 'Cm', 'Ch', 'VV', 'Td',
       'RRR', 'tR', 'E', 'Tg', 'E'', 'sss', 'station_id'],
      dtype='object')

In [13]:
weather_train[weather_train.station_id == 34123]

,local_time,T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Ch,VV,Td,RRR,tR,E,Tg,E',sss,station_id
0,31.12.2013 21:00,-1.5,758.9,773.2,0.7,99.0,"Ветер, дующий с северо-северо-запада",2.0,NaN,NaN,...,NaN,6.0,-1.6,NaN,NaN,NaN,NaN,NaN,NaN,34123
1,31.12.2013 18:00,-1.7,758.2,772.5,0.7,99.0,"Ветер, дующий с северо-запада",3.0,NaN,NaN,...,NaN,3.0,-1.9,Осадков нет,12.0,NaN,NaN,NaN,NaN,34123
2,31.12.2013 15:00,-1.7,757.5,771.8,0.0,99.0,"Ветер, дующий с северо-запада",4.0,NaN,NaN,...,NaN,3.0,-1.9,NaN,NaN,NaN,NaN,NaN,NaN,34123
3,31.12.2013 12:00,-1.2,757.5,771.7,0.8,99.0,"Ветер, дующий с северо-запада",3.0,NaN,NaN,...,NaN,2.5,-1.4,NaN,NaN,NaN,NaN,NaN,NaN,34123
4,31.12.2013 09:00,-1.2,756.7,770.9,1.2,99.0,"Ветер, дующий с юга",3.0,NaN,NaN,...,NaN,0.5,-1.4,NaN,NaN,NaN,NaN,NaN,NaN,34123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51738,01.01.2014 12:00,-1.0,760.6,775.0,0.1,95.0,"Ветер, дующий с юго-запада",1.0,NaN,NaN,...,NaN,10.0,-1.7,NaN,NaN,NaN,NaN,NaN,NaN,34123
51739,01.01.2014 09:00,-1.3,760.5,774.8,0.5,96.0,"Штиль, безветрие",0.0,NaN,NaN,...,NaN,10.0,-1.8,NaN,NaN,NaN,NaN,NaN,NaN,34123
51740,01.01.2014 06:00,-1.2,760.0,774.3,0.0,95.0,"Ветер, дующий с северо-запада",1.0,NaN,NaN,...,NaN,10.0,-1.9,Осадков нет,12.0,NaN,NaN,Слежавшийся или мокрый снег (со льдом или без ...,1.0,34123
51741,01.01.2014 03:00,-1.2,760.0,774.3,0.4,97.0,"Ветер, дующий с западо-северо-запада",1.0,NaN,NaN,...,NaN,10.0,-1.6,NaN,NaN,NaN,NaN,NaN,NaN,34123
